In [8]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
import os
import pandas as pd
import numpy as np

In [9]:
TRAIN_DIR = 'images/train'
TEST_DIR = 'images/test'

In [14]:
def createdataframe(dir):
    image_paths = []
    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

In [15]:
train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)
print(train)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                                image     label
0            images/train\angry\0.jpg     angry
1            images/train\angry\1.jpg     angry
2           images/train\angry\10.jpg     angry
3        images/train\angry\10002.jpg     angry
4        images/train\angry\10016.jpg     angry
...                               ...       ...
28816  images/train\surprise\9969.jpg  surprise
28817  images/train\surprise\9985.jpg  surprise
28818  images/train\surprise\9990.jpg  surprise
28819  images/train\surprise\9992.jpg  surprise
28820  images/train\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [18]:
test = pd.DataFrame()
test['image'], test['label'] = createdataframe(TEST_DIR)
print(test)

angry completed
disgust completed
fear completed
happy completed
neutral completed
sad completed
surprise completed
                               image     label
0            images/test\angry\0.jpg     angry
1            images/test\angry\1.jpg     angry
2           images/test\angry\10.jpg     angry
3        images/test\angry\10002.jpg     angry
4        images/test\angry\10016.jpg     angry
...                              ...       ...
28816  images/test\surprise\9969.jpg  surprise
28817  images/test\surprise\9985.jpg  surprise
28818  images/test\surprise\9990.jpg  surprise
28819  images/test\surprise\9992.jpg  surprise
28820  images/test\surprise\9996.jpg  surprise

[28821 rows x 2 columns]


In [19]:
def extract_features(images):
    features = []
    for image in tqdm(images):
        img = load_img(image, grayscale=True)
        img = np.array(img)
        features.append(img)
    features = np.array(features)
    features = features.reshape(len(features), 48, 48, 1)
    return features

In [20]:
from tqdm.notebook import tqdm

In [21]:
train_features = extract_features(train['image'])
test_features = extract_features(test['image'])

  0%|          | 0/28821 [00:00<?, ?it/s]

C:\Users\Admin\AppData\Local\Programs\Python\Python310\lib\site-packages\keras_preprocessing\image\utils.py:107: UserWarning: grayscale is deprecated. Please use color_mode = "grayscale"
  warnings.warn('grayscale is deprecated. Please use '


  0%|          | 0/28821 [00:00<?, ?it/s]

In [22]:
x_train = train_features / 255.0
x_test = test_features / 255.0

In [23]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_test = le.transform(test['label'])
y_train = to_categorical(y_train, num_classes=7)
y_test = to_categorical(y_test, num_classes=7)

In [24]:
# sequential model
model = Sequential()

# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# flattening
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))

# output layer
model.add(Dense(7, activation='softmax'))

# model compilation
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [ ]:
model.fit(x=x_train, y=y_train, batch_size=128, epochs=100, validation_data=(x_test, y_test))

Epoch 1/100
226/226 [==============================] - 1362s 6s/step - loss: 1.8054 - accuracy: 0.2503 - val_loss: 1.7824 - val_accuracy: 0.2611
Epoch 2/100
226/226 [==============================] - 2120s 9s/step - loss: 1.7536 - accuracy: 0.2735 - val_loss: 1.6638 - val_accuracy: 0.3346
Epoch 3/100
226/226 [==============================] - 6306s 28s/step - loss: 1.6237 - accuracy: 0.3537 - val_loss: 1.4627 - val_accuracy: 0.4279
Epoch 4/100
226/226 [==============================] - 1801s 8s/step - loss: 1.5077 - accuracy: 0.4130 - val_loss: 1.3678 - val_accuracy: 0.4684
Epoch 5/100
226/226 [==============================] - 76508s 340s/step - loss: 1.4471 - accuracy: 0.4397 - val_loss: 1.2898 - val_accuracy: 0.5048
Epoch 6/100
226/226 [==============================] - 1008s 4s/step - loss: 1.4045 - accuracy: 0.4605 - val_loss: 1.2730 - val_accuracy: 0.5141
Epoch 7/100
226/226 [==============================] - 985s 4s/step - loss: 1.3678 - accuracy: 0.4700 - val_loss: 1.2206 - val

In [ ]:
model_json = model.to_json()
with open("emotiondetector.json", 'w') as json_file:
    json_file.write(model_json)
model.save("emotiondetector.h5")